In [1]:
keywords_list = ['Russians', 'Russian', 'Russia', 'Soviet', 'USSR', 'Stalin', 'Moscow']

In [3]:
# присваеваем значение для переменной, определяющей "шаг" интервала времени, в днях
delta = 29

In [4]:
# импортируем библиотеку NYTAPI с помощью которой будем обращаться к электронному архиву Нью-Йорк Таймс через API (https://github.com/michadenheijer/pynytimes/tree/0.8.0)
from pynytimes import NYTAPI
import datetime as datetime

# Import required libraries
import time
import pandas as pd
from datetime import datetime as datetime_2

# ключ API, полученный в личном кабинете на сайте NYT
api_key = "<your_key_value_here>"

# инициализируем NYTAPI, применяя свой ключ, полученный в личном кабинете (инструкции здесь: https://developer.nytimes.com/get-started)
nyt = NYTAPI(api_key, parse_dates=True)

# задаем начало и конец периода для поиска статей именно в этом интервале за "шаг"
begin = datetime_2(1941, 6, 22) 
end = begin + datetime.timedelta(days=delta)
# создаем словарь со значениями нашего окна поиска по датам
date_dict = {"begin": begin, "end": end}

# дополнительный словарь где определяются дополнительные параметры посика. Все параметры закомментены, корме опции сортировки результатов от старых к новым
options_dict = {
    # Sort from earliest to latest
    "sort": "oldest",
    # Return only articles from New York Times (filters out other sources such as AP and Reuters)
    #"sources": ["New York Times"],
    # Return only news, analyses, and articles
    #"type_of_material": ["News Analysis", "News", "Article"]
}


In [5]:
articles = []
# ограничиваем кол-во шагов в цикле (кол-во окон по датам) 2 циклами. Подобрано экспериментальным путем. Если больше, то цикл не завершается, упираясь в лимиты NYT по кол-ву запросов API в сутки 
for i in range(2):
    for key_word in keywords_list:
        articles_for_cycle = nyt.article_search(
            query = key_word,
            results=400,
            dates=date_dict,
            options=options_dict)
        for item in articles_for_cycle:
            if item not in articles:
                articles.append(item)
                
        # выводим кол-во статей для каждого ключевого слова из списка
        print(key_word + ": Length of list with aticles is " + str(len(articles)))
    # выводим кол-во статей после завершения поиска для каждого окна дат
    print(str(i+1) + " step (" + str(begin) + " - " + str(end) + "). " + "Rows in the list of articles: " + str(len(articles)) + " rows")

    # смещаем окно поиска по датам на delta
    begin += datetime.timedelta(days=delta+1)
    end += datetime.timedelta(days=delta+1)
    date_dict = {"begin": begin,  "end": end}
    
    # пауза 5 сек перед следующим обращением
    time.sleep(5)

C:\ProgramData\anaconda3\Lib\site-packages\pynytimes\helpers\article_search.py:33: UserWarning: Asking for a lot of results, because of rate limits it can take a while.
  warnings.warn(


In [13]:
final_articles = articles[:]

In [15]:
# преобразуем json в dataframe 
df = pd.json_normalize(final_articles)

In [ ]:
# сохраняем DataFrame в таблицу формата csv  (изенить параметр на HEADER = TRUE если это  первый цикл и таблица создается впервые)
df.to_csv('RAW final_7_key_words(with tab delimeter).csv', sep='\t', encoding='utf-8', mode='a', header=True)

In [17]:
# здесь оставляем только интрересующие нас поля для дальнейшего анализа
df = df.loc[:,['abstract','web_url','lead_paragraph', 'print_section', 'print_page', 'keywords', 'pub_date', 'news_desk', 'type_of_material', '_id', 'word_count', 'headline.main', 'headline.kicker', 'headline.print_headline', 'byline.original', 'byline.person', 'byline.organization']]

In [18]:
# оцениваем верхнеуровнево качесвто данных, а именно сколько пустых значений в полях
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1590 entries, 0 to 1589
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   abstract                 1590 non-null   object             
 1   web_url                  1590 non-null   object             
 2   lead_paragraph           1590 non-null   object             
 3   print_section            279 non-null    object             
 4   print_page               1587 non-null   object             
 5   keywords                 1590 non-null   object             
 6   pub_date                 1590 non-null   datetime64[ns, UTC]
 7   news_desk                1590 non-null   object             
 8   type_of_material         1590 non-null   object             
 9   _id                      1590 non-null   object             
 10  word_count               1590 non-null   int64              
 11  headline.main            1590 

In [19]:
# сохраняем таблицу в формате csv
df.to_csv('final_7_key_words(with tab delimeter).csv', sep='\t', encoding='utf-8', mode='a', header=True)